In [1]:
import fiona
import numpy as np
import pyproj
import shapely.geometry as sgeom
import pathlib
import zipfile

<ipython-input-1-b79608866b2d>:5: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [3]:
import json
from io import StringIO

from pygridder import pygridder as pgrid

### Ensuring we can load ndfd file

In [4]:
ndfd_file = '../../dev/impacts-app/scripts/impacts-data/orig-gis-data/ndfd.npz'

In [5]:
with np.load(ndfd_file) as NPZ:
    X = NPZ['X']
    Y = NPZ['Y']
    proj = pyproj.Proj(NPZ['srs'].item())

In [13]:
G = pgrid.Gridder(X,Y,dx=4000)

In [14]:
hosp_file = '../../dev/impacts-app/scripts/impacts-data/orig-gis-data/hospitals.zip'

In [15]:
zipped = zipfile.ZipFile(hosp_file)

In [18]:
zipped.extractall('./example_files/')

In [19]:
shp_file = './example_files/hospitals/hospitals.shp'

In [21]:
with fiona.open(shp_file) as c:
    print(len(c))

7496


### Reading & Gridding US Public schools file

In [22]:
schools_file = './example_files/Public_Schools-shp.zip'

In [23]:
zipped = zipfile.ZipFile(schools_file)

In [24]:
zipped.extractall('./example_files/')

In [25]:
shp_file = './example_files/PublicSchools.shp'

In [61]:
schools = G.make_empty_grid()

In [62]:
slons = []
slats = []

with fiona.open(shp_file) as SHP:
    for shp in SHP:
        props = shp['properties']
        
        if props['LONGITUDE'] > -127.37 and props['LONGITUDE'] < -67.31:
            slons.append(props['LONGITUDE'])
            slats.append(props['LATITUDE'])
        

In [63]:
slons, slats = proj(slons,slats)

In [64]:
points = G.grid_points(slons,slats)

In [65]:
for point in points:
    schools[point] += 1   

In [66]:
np.max(schools)

230

### Reading & Gridding US Private School Files

In [68]:
pvschools_file = './example_files/Private_Schools-shp.zip'

In [69]:
zipped = zipfile.ZipFile(pvschools_file)
zipped.extractall('./example_files/')

In [73]:
shp_file = './example_files/Private_Schools.shp'

In [71]:
pvschools = G.make_empty_grid()

In [74]:
slons = []
slats = []

with fiona.open(shp_file) as SHP:
    for shp in SHP:
        props = shp['properties']
        
        if props['LONGITUDE'] > -127.37 and props['LONGITUDE'] < -67.31:
            slons.append(props['LONGITUDE'])
            slats.append(props['LATITUDE'])

In [75]:
slons, slats = proj(slons,slats)

In [76]:
points = G.grid_points(slons,slats)

In [77]:
for point in points:
    pvschools[point] += 1   

In [80]:
tot_schools = schools + pvschools

In [82]:
np.savez_compressed('./schools.npz',schools=tot_schools)

### Add to impact-grids.npz

In [88]:
temp_dict = {}

In [83]:
impact_grids = np.load('../scripts/pas-input-data/impact-grids.npz')

In [89]:
for key in impact_grids.keys():
    temp_dict[key] = impact_grids[key]

In [95]:
temp_dict['schools'] = tot_schools

In [96]:
np.savez_compressed('./impact-grids-new.npz',**temp_dict)